# Importing all the necessary Libraries

In [ ]:
from nltk.corpus import reuters 
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import spacy
import numpy as np

'''
  https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html
  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
  https://spacy.io/usage/vectors-similarity
'''

'\n  https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html\n  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html\n  https://spacy.io/usage/vectors-similarity\n'

In [ ]:
import nltk
nltk.download('reuters')
!python -m spacy download en_core_web_lg

[nltk_data] Downloading package reuters to /root/nltk_data...
     |████████████████████████████████| 827.9MB 1.3MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180945 sha256=7a8fc23fed4ae020777c1c624b6ca4673d567ea3b89aa86cc7253ea9851b743f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lh804lp4/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Loading Data

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
def collection_stats():
  documents = reuters.fileids()
  print(str(len(documents)) + " documents");

  train_docs = list(filter(lambda doc: doc.startswith("train"), documents));
  print(str(len(train_docs)) + " total train documents");
 
  test_docs = list(filter(lambda doc: doc.startswith("test"), documents));
  print(str(len(test_docs)) + " total test documents")

  categories = reuters.categories()

  print(str(len(categories)) + " categories");

In [ ]:
collection_stats()

10788 documents
7769 total train documents
3019 total test documents
90 categories


# Train Test Split of Data

In [ ]:
def train_test_split():
  documents = reuters.fileids()
  train_docs = [document for document in documents if document.startswith("train")]
  test_docs = [document for document in documents if document.startswith("test")]
  x_train = [reuters.raw(doc_id) for doc_id in train_docs]
  y_train = [reuters.raw(doc_id) for doc_id in test_docs]
  x_test = mlb.fit_transform([reuters.categories(doc_id) for doc_id in train_docs])
  y_test = mlb.transform([reuters.categories(doc_id) for doc_id in test_docs])
  return x_train, y_train, x_test, y_test

In [ ]:
x_train, x_test, y_train, y_test = train_test_split()

# Data Preprocessing

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_text(X_train):
  clean =[]
  for term in X_train:
    term=re.sub(r'https\S+' , '' ,term)
    term=re.sub('[^a-zA-Z]' , ' ', term)
    term = str(term).lower()
    term=word_tokenize(term)
    term=[item for item in term if item not in stop_words]
    term =' '.join(term)
    clean.append(term)
  return clean
  

In [ ]:
X_train=clean_text(x_train)
X_test =clean_text(x_test)

# Word2Vec Representation

In [ ]:
nlp = spacy.load("en_core_web_lg")


In [ ]:
def get_word_vectors(sentence):
  tokens = nlp(sentence)
  vector=np.sum([token.vector for token in tokens] ,axis =0)
  return vector

## Generate Word2Vec embeddings for training data

In [ ]:
get_word_vectors(X_train[0]).shape

(300,)

In [ ]:
x_word2vec_train=[get_word_vectors(doc) for doc in X_train]

In [ ]:
np.shape(x_word2vec_train)

(7769, 300)

## Generate Word2Vec embeddings for testing data

In [ ]:
x_word2vec_test=[get_word_vectors(doc) for doc in X_test]

In [ ]:
np.shape(x_word2vec_test)

(3019, 300)

# Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

## Fit and Predict Model

In [ ]:
lr=OneVsRestClassifier(LogisticRegression(solver='newton-cg'))
lr.fit(x_word2vec_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='newton-cg', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
y_pred1 = lr.predict(x_word2vec_test)

## Classification report on training data

In [ ]:
#not needed
#print("Word2vec Result word on Train")
3print(classification_report( ovr.predict(X_train) , y_train))

## Classification report on testing data

In [ ]:
print("Word2vec Result word on Test")
print(classification_report( lr.predict(x_word2vec_test) , y_test))

Word2vec Result word on Test
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       702
           1       0.30      0.78      0.44         9
           2       0.71      0.83      0.77        12
           3       0.63      0.73      0.68        26
           4       0.78      0.67      0.72        21
           5       0.00      0.00      0.00         0
           6       0.78      0.70      0.74        20
           7       0.50      1.00      0.67         1
           8       0.00      0.00      0.00         1
           9       0.93      0.79      0.85        33
          10       0.78      0.82      0.80        17
          11       0.00      0.00      0.00         1
          12       0.79      0.65      0.71        68
          13       0.55      0.69      0.61        16
          14       0.00      0.00      0.00         0
          15       0.64      0.67      0.65        27
          16       0.00      0.00      0.00         

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier

## Fit and Predict Model

In [ ]:
gnb = OneVsRestClassifier(GaussianNB())
gnb.fit(x_word2vec_train,y_train)

OneVsRestClassifier(estimator=GaussianNB(priors=None, var_smoothing=1e-09),
                    n_jobs=None)

In [ ]:
y_pred2=gnb.predict(x_word2vec_test)

## Classification report on training data

In [ ]:
#print("Naive Bayes Classifier Result word on Train")
#print(classification_report(nbClassifier.predict(X_train) , y_train))

## Classification report on testing data

In [ ]:
print("Naive Bayes Classifier Result word on Test")
print(classification_report(gnb.predict(x_word2vec_test) , y_test))

Naive Bayes Classifier Result word on Test
              precision    recall  f1-score   support

           0       0.92      0.34      0.50      1918
           1       0.70      0.03      0.05       640
           2       0.86      0.05      0.09       260
           3       0.47      0.04      0.08       319
           4       0.78      0.09      0.17       149
           5       0.00      0.00      0.00         0
           6       0.28      0.02      0.03       305
           7       0.00      0.00      0.00        38
           8       0.67      0.10      0.17        20
           9       0.36      0.03      0.06       333
          10       0.72      0.02      0.04       631
          11       0.00      0.00      0.00         0
          12       0.43      0.07      0.12       331
          13       0.65      0.08      0.15       154
          14       0.00      0.00      0.00         0
          15       0.57      0.12      0.19       137
          16       1.00      1.00     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
